In [1]:
from transformers import pipeline, set_seed
import pandas as pd
import requests, sys, webbrowser,xml
import numpy as np
import bs4
import re
import random

### Pipeline usage to perform test generation, using the hugging face package transformers. Citation is below:
@article{Wolf2019HuggingFacesTS,
  title={HuggingFace's Transformers: State-of-the-art Natural Language Processing},
  author={Thomas Wolf and Lysandre Debut and Victor Sanh and Julien Chaumond and Clement Delangue and Anthony Moi and Pierric Cistac and Tim Rault and R'emi Louf and Morgan Funtowicz and Jamie Brew},
  journal={ArXiv},
  year={2019},
  volume={abs/1910.03771}
}

In [2]:
# Starting up pipeline
def start_pipeline():
    generator = pipeline('fill-mask', model='bert-base-uncased')
    return generator

In [3]:
nlp = start_pipeline()

/Users/gilpasternak/opt/miniconda3/lib/python3.7/site-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

In [4]:
# Given a number letters and some sequence, returns the sequence with generated attached words
def generate_text(generator,intro_sequence:str, num_words = 5)->str:
    # Imported random text generation
    text = generator(intro_sequence, max_length=len(intro_sequence) + num_words, num_return_sequences=1)[0].get("generated_text")
    return text

In [5]:
def get_text(url = "https://www.keepinspiring.me/famous-quotes/" ):
    # Requesting data from url, finding specialized tags for this particular website
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    text  = soup.find_all("div", class_ = 'author-quotes')  
    return text

In [9]:
text = get_text()

## Game List: 

*Format: select the game -> select the text type*

1) **Mad Libs** 
<ul>
    <li>Favorite song Edition</li>
    <li>Wikipedia edition</li>
    <li>All time famous quotes edition</li>
    <li>Generate a normal mad-lib using sentiment analysis to get parts of speech, tell user in random 'noun,verb, etc.'</li>
</ul>

2) **Find the real text**
<ul>
    <li>Favorite song Edition</li>
    <li>Wikipedia edition</li>
    <li>All time famous quotes edition</li>
</ul>

3) **How well do you know your favorite song?**

4) **Can you guess what Wikipedia page this used to be?**

## Word Processing Below
Scraping the data from our quote website and cleaning it up

In [10]:
# Obtainin author from quote
def authors(quote):
    return quote.split("”")[1]

#Removing author and adding lost smartquote
def remove_authors(quote):
    return (quote.split("”")[0] + ("”"))

In [11]:
#Tag processing functions to clean up nasty html formatting, replaces div tags
def processing_div(tag):
    return tag.replace('<div class="author-quotes">', "").replace("</div>", "")

#span processing, replaces span tag
def processing_span(tag):
    return tag.replace("<span class=\"quote-author-name\">", "").replace("</span>", "")

# Checks for tags that have yet to be removed, not given standard format. Reasoning - we don't know when ads will pop up
def cleaner(table):
    arr = np.array([])
    for i in table.get("quote"):
        arr = np.append(arr,("<" in i))
    clean = table[(arr != 1)]
    return clean

In [12]:
table = pd.DataFrame().assign(quote = text)

#Formatting, processing, and splitting quotes and authors
def table_process(table):
    table = table.assign(quote = table.get("quote").apply(str))
    table = table.assign(quote = table.get("quote").apply(processing_div).apply(processing_span))
    table = cleaner(table)
    table = table.assign(author  = table.get("quote").apply(authors), quote = table.get("quote").apply(remove_authors))
    return table
table = table_process(table)
table

,quote,author
0,“You know you’re in love when you can’t fall a...,– Dr. Suess
1,"“I’m selfish, impatient and a little insecure....",– Marilyn Monroe
2,“Get busy living or get busy dying.”,– Stephen King
3,“The first step toward success is taken when y...,– Mark Caine
5,“Twenty years from now you will be more disapp...,– Mark Twain
6,“When I dare to be powerful – to use my streng...,– Audre Lorde
8,“A successful man is one who can lay a firm fo...,– David Brinkley
10,“I can’t give you a sure-fire formula for succ...,-Herbert Bayard Swope
11,“Would you like me to give you a formula for s...,– Thomas J. Watson
12,"“It is hard to fail, but it is worse never to ...",– Theodore Roosevelt


## Random Word Replacement to be used in each individual Turing game

In [13]:
def replace_words_at_random(generator, word_arr: [str], num_words: int, difficulty = 1) -> ([str],[int]):
    
    #  Generate num_words random indices
    indices = random_generation(num_words, 0, len(word_arr))
    print(indices)
    
    # Place word masks at each of the randomly chosen indices
    # Fill in each mask with the language model
    for i in indices:
        #Ensuring array is not overaccessed
        if(i < len(word_arr)):
            word_arr[i] = '[MASK]'
        
        #Adding together everything leading up to string so as to add some context into the model
        join  = " ".join(word_arr)

        # Generate the next word
        text = (generator(join)[3-difficulty].get('sequence').replace("[CLS]", '').replace('[SEP]', '')).strip()
        word_arr = text.split(" ")

    return text, indices 

# Generates a desired number of unique random digits in a certain range
def random_generation(num_words, lowNum, highNum):
    random_digits = np.unique(np.random.randint(low = lowNum, high  = highNum, size = num_words))
    while(len(random_digits) < num_words):
        random_digits = np.append(random_digits, np.random.randint(low = lowNum, high = highNum, size = num_words-len(random_digits)))
        random_digits = np.unique(random_digits)
    return random_digits

## Wikipedia Search Below

Start at user given wikipedia page, randomly click a certain number of links from there and scrape 5 sentences from the final page

In [14]:
# Obtains text from a wikipedia url
def get_wiki_text(url):
    # Requesting data from url, finding specialized tags for this particular website
    res = requests.get(url)
    res.raise_for_status()
    
    #Attaching soup object to page text, obtaining text in paragraphs
    soup = bs4.BeautifulSoup(res.text, "lxml")
    text = ""
    
    # Problematic structure: fails to look for list items which make up substantial amount of wikipedia pages
    for paragraph in soup.find_all('p'):
        text+= paragraph.text
        
    # Formatting the string so that it looks normal
    text = re.sub(r'\[.*\]', '', text)
    text= re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [15]:
# Iterates through wikipedia pages
def wiki_search():
    topic, pages = inp()
    textSoup, title = looping_wiki_search(topic, int(pages))
    return textSoup, title 

# Randomly selecting the next topic to be searched for
def select_next_topic(text: [str])-> str:
    random_number = np.random.randint(1,len(text))
    return text[random_number]

# Replaces hyphens with an underscore for url purposes, removes all punctuation that could break url
def replace_punctuation(text):
    specific_case = text.strip()
    specific_case = specific_case.replace('-',' ')
    specific_case = specific_case.replace(' ', '_')
    
    # Remove all punctuation
    pattern = re.compile(r'\W')
    specific_case = re.sub(pattern, '', specific_case)
    
    return specific_case
    
    
# Loop through connected topics on wikipedia to find a "landing page", then return the text of that landing page. By default, returns the page of 'topic'
def looping_wiki_search(topic: str, neighbor_pages = 0):
    searchText = ""
    url = construct_wiki_url(topic)
    for i in np.arange(neighbor_pages+1):
        print(i)
        searchText = get_wiki_text(url).split(" ")
        
        
        #So long as there are still pages left to proccess
        if(i < neighbor_pages):
            
            # Selecting next topic
            topic = select_next_topic(searchText)

            # Replacing the punctuation of the next topic
            topic = replace_punctuation(topic)

            # Moving to next URL
            url = construct_wiki_url(topic)
        
    return searchText, topic
        

# Temporary dummy input
def inp():
    print("Please enter a topic")
    val = str(input())
    print('Please enter the number of neighbor pages: ')
    pages = input()
    return val, pages

# Takes a given user topic and constructs a valid wikipedia url
def construct_wiki_url(url_topic : str):
    return ('https://en.wikipedia.org/wiki/' + url_topic.lower().strip().replace(" ","_"))

In [22]:
wiki_search()

Please enter a topic


 zeus


Please enter the number of neighbor pages: 


 3


0
1
2
3


(['as,', 'as,', 'a/s', 'or', 'similar', 'may', 'refer', 'to:', ''], 'as')

## Last Scrape for Song Game: Scraping Genius

In [16]:
import lyricsgenius

In [17]:
# Takes artist and song name in terms of string and obtains lyrics
def obtain_lyrics(artist: str, song: str) -> [str]:
    genius = lyricsgenius.Genius("NDltUrlbSis8n9o1FEyGUE_ruIlngdDmXwoQdvrkX0hh3le3LKF8XalcHXOetm3x")
    artist = genius.search_artist(artist, max_songs=3, sort="title")
    song = genius.search_song(song, artist.name)
    song_lyrics =  song.lyrics
    
    #String proccessing
    song_lyrics = re.sub(r'\[.*\]', '', song_lyrics)
    song_lyrics  = re.sub(r'\n+', ' ', song_lyrics)
    song_lyrics = song_lyrics.split(" ")
    
    #Making sure songs do not exceed maximum threshold
    if(len(song_lyrics) > 510):
        song_lyrics = song_lyrics[:510]
    return song_lyrics

# Master Controls obtaining and proccessing lyrics
# To do: preserve tags, ie [verse 1], and keep word replacement model from touching it 
def lyrics(artistName, songName):
    lyrics = obtain_lyrics(artistName, songName) 
    return lyrics

In [20]:
#text, indices = replace_words_at_random(nlp, obtain_lyrics('Yung Gravy', '1 Thot 2 Thot Red Thot Blue Thot'), 90, -1) # Once this runs, we celebrate
text, indices = replace_words_at_random(nlp, obtain_lyrics('Queen', 'We are the champion'), 20, 2)

Searching for songs by Queen...

Song 1: "13"
Song 2: "20th Century Fox Fanfare"
Song 3: "’39"

Reached user-specified song limit (3).
Done. Found 3 songs.
Searching for "We are the champion" by Queen...
Done.
[  6  24  26  29  32  49  52  70  73  75  92  96 117 131 134 163 166 179
 189 196]


In [29]:
# Controller for the madlib game, pulls from different data sources to create madlibs based off type
def madlib_control(text_type):
    pass
    
# Controller for identify original text, should return original and modified text in random order (option to return multiple?)
def identify_original_text(text_type)
    pass
    
def guess_the_page():
    pass


i heard the drums echoing tonight but she hears only whispers of some quiet conversation she's coming in, 12 : 30 flight her moonlit wings reflect the stars that guide me towards salvation i stopped an old man along the way hoping to find some old forgotten words or ancient melodies he turned to me as if to say " hurry boy, it's waiting there for you " it's gonna take a lot to drag me away for you there's nothing that a hundred men or more could ever do i bless those rains down in africa gonna take some time to do the things we never had the wild dogs cry out in the night as they grow restless longing for some solitary company i know that i must do what's right as sure as kilimanjaro rises like olympus above the serengeti i seek to cure what's deep inside frightened of this thing that i've become it's gonna take a lot to drag me back to you there's nothing that a hundred men or more can ever do i bless the rain down in africa gonna take some time to do the things we never had hurry boy

In [21]:
text

"i've paid my dues time to time i've done my sentence but committed no crime and bad mistakes i've made a few i've had our share the sand kicked from my face but i've come through ( and i need to go on and on, and on, and on. we are the winners my friends and we'll keep on fighting till the end we are the champions we are the winners no time to losers'cause they are the champions of the world i've taken my bows and my curtain calls you brought my fame and fortune, everything that goes with it i thank you all but it's been no bed of roses no pleasure cruise i consider it a challenge before the whole human race and i ain't gonna lose ( so i need not go on and on, and on, and on ) we are the champions, my friends and we'll keep on fighting till the end we are the champions we were the champions last time for losers'cause we are the champions of the world we are all champions, my friends and we'll keep on fighting till an end, oh we are the champions they are the champions no time for lose